### A Simple Function

In [13]:
import json

In [14]:
def get_current_weather(location: str, unit: str):
    """

    """
    if location == "Hanoi":
        return json.dumps({"Temperature": 20, "unit": unit})
    else:
        return json.dumps({"Temperature": 50, "unit": unit})

In [15]:
get_current_weather("Hanoi", unit="celsius")

'{"Temperature": 20, "unit": "celsius"}'

In [16]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
Bạn là một mô hình AI gọi hàm. Bạn được cung cấp các chữ ký hàm trong thẻ XML <tools></tools>.
Bạn có thể gọi một hoặc nhiều hàm để hỗ trợ truy vấn của người dùng. Đừng đưa ra giả định về giá trị cần truyền vào hàm.
Hãy đặc biệt chú ý đến thuộc tính 'types'. Bạn nên sử dụng các kiểu dữ liệu này giống như trong một từ điển Python.
Đối với mỗi lần gọi hàm, hãy trả về một đối tượng JSON với tên hàm và các đối số trong thẻ XML <tool_call></tool_call> như sau:
<tool_call>

{"name": <tên hàm>,"arguments": <từ điển đối số>}

</tool_call>

Đây là các công cụ có sẵn:

<tools> {
    "name": "get_current_weather",
    "description": "Lấy thời tiết hiện tại tại một vị trí nhất định location (str): Thành phố và tiểu bang, ví dụ: Hanoi, Danang unit (str): Đơn vị. Nó có thể nhận hai giá trị; 'độ C', 'độ F',
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

In [17]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current temperature in Hanoi, in Celsius?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)


<tool_call>
{
    "name": "get_current_weather",
    "arguments": {
        "location": "Hanoi",
        "unit": "độ C"
    }
}
</tool_call>


In [18]:
%cd ..

/home/ngocson/NSN/Project


In [29]:
import json
import requests
from src.agentic_patterns.tool_pattern.tool import *
from src.agentic_patterns.tool_pattern.tool_agent import ToolAgent

In [30]:
import json
import requests
from src.agentic_patterns.tool_pattern.tool import tool
from src.agentic_patterns.tool_pattern.tool_agent import ToolAgent

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. 
    Each story contains the title, URL, score, author, and time of submission. The data is fetched 
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    
    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Get the top story IDs
        top_story_ids = response.json()[:top_n]
        
        top_stories = []
        
        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()
            
            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })
        
        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return [] 

In [31]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'Beginning January 2026, all ACM publications will be made open access',
  'url': 'https://dl.acm.org/openaccess'},
 {'title': '1.5 TB of VRAM on Mac Studio – RDMA over Thunderbolt 5',
  'url': 'https://www.jeffgeerling.com/blog/2025/15-tb-vram-on-mac-studio-rdma-over-thunderbolt-5'},
 {'title': 'History LLMs: Models trained exclusively on pre-1913 texts',
  'url': 'https://github.com/DGoettlich/history-llms'},
 {'title': 'We pwned X, Vercel, Cursor, and Discord through a supply-chain attack',
  'url': 'https://gist.github.com/hackermondev/5e2cdc32849405fff6b46957747a2d28'},
 {'title': 'Texas is suing all of the big TV makers for spying on what you watch',
  'url': 'https://www.theverge.com/news/845400/texas-tv-makers-lawsuit-samsung-sony-lg-hisense-tcl-spying'}]

In [32]:
hn_tool = tool(fetch_top_hacker_news_stories)

In [33]:
tool_agent = ToolAgent(tools=[hn_tool])

In [37]:
result = tool_agent.run(user_msg="1+ 5 bằng mấy?")

In [38]:
print(result)

1 + 5 = 6
